In [ ]:
%reload_ext autoreload
%autoreload 2

import os, sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2
import ast
from tqdm import tqdm
from collections import defaultdict

In [ ]:
# install these
#https://pymicro.readthedocs.io/projects/pymicro/en/latest/cookbook/pointset_registration.html
from pymicro.view.vol_utils import compute_affine_transform
#https://pypi.org/project/affine6p/
import affine6p
#https://github.com/jewettaij/superpose3d
from superpose3d import Superpose3D

In [ ]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer, \
    create_full_volume, get_structure_colors
from utilities.sqlcontroller import SqlController
animal = 'MD589'
sqlController = SqlController(animal)

CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, 'MD589_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations.head()
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))

hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))
structures = ['DC', 'LC','SC', '5N', '7n']
# SC is the only singular structure
df = hand_annotations[['name', 'section', 'side', 'vertices']].copy()
df = df.loc[df['name'].isin(structures)]
df = df.reset_index()
df = df.sort_values(by=['name', 'section'])
df.head()

In [ ]:
# these are the 5 structures Beth suggested for getting
# enough that were spread out.
structures = ['DC', 'LC','SC', '5N', '7n']
centers = {}
section_structure_vertices = defaultdict(dict)
SC = df.loc[(df['name'] == "SC")]
first_section = SC['section'].min()
last_section = SC['section'].max()
midsection = int(round(((last_section - first_section) / 2) + first_section))
point_array = SC['vertices'].loc[SC['section'] == midsection].values
section_structure_vertices[midsection]['SC'] = point_array[0]
cx, cy = np.mean(point_array[0], axis=0)    
centers['SC'] = [int(round(cx)), int(round(cy)), int(round(midsection))]

structures.remove('SC')
for structure in structures:
    df_tmp = df.loc[(df['name'] == structure) & (df['side'] == "L")]
    first_section = df_tmp['section'].min()
    last_section = df_tmp['section'].max()
    midsection = int(round(((last_section - first_section) / 2) + first_section))
    structure_name = structure + "_L"
    point_array = df_tmp['vertices'].loc[df_tmp['section'] == midsection].values
    section_structure_vertices[midsection][structure_name] = point_array[0]
    cx, cy = np.mean(point_array[0], axis=0)    
    centers[structure_name] = [int(round(cx)), int(round(cy)), int(round(midsection))]
    
    df_tmp = df.loc[(df['name'] == structure) & (df['side'] == "R")]
    first_section = df_tmp['section'].min()
    last_section = df_tmp['section'].max()
    midsection = int(round(((last_section - first_section) / 2) + first_section))
    structure_name = structure + "_R"
    point_array = df_tmp['vertices'].loc[df_tmp['section'] == midsection].values
    section_structure_vertices[midsection][structure_name] = point_array[0]
    cx, cy = np.mean(point_array[0], axis=0)    
    centers[structure_name] = [int(round(cx)), int(round(cy)), int(round(midsection))]


In [ ]:
centers

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
thickness = 1
colors = {}
colors['SC'] = (255,0,255)

colors['DC_L'] = (0,0,0)
colors['LC_L'] = (255,0,0)
colors['SC_L'] = (255,0,255)
colors['5N_L'] = (0,255,255)
colors['7n_L'] = (100,10,255)
colors['DC_R'] = (0,0,0)
colors['LC_R'] = (255,0,0)
colors['SC_R'] = (255,0,255)
colors['5N_R'] = (0,255,255)
colors['7n_R'] = (100,10,255)
# create a fake random distance away from the original structures
rand_offset = np.random.randint(75,150)
offsets = {}
section_images = {}
points = {}
SCALE = 32
PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1'
thumbnail_dir = os.path.join(PATH, 'thumbnail')
files = sorted(os.listdir(thumbnail_dir))
for file_name in tqdm(files):
    filepath = os.path.join(thumbnail_dir, file_name)
    img = cv2.imread(filepath)
    section = int(file_name.split('.')[0])
    
    for structure in section_structure_vertices[section]:
        pts = section_structure_vertices[section][structure]
        points = np.array(pts, dtype=np.int32)        
        points = points // SCALE
        
        points_diff = np.rint(points + rand_offset)
        points_diff = points_diff.astype(np.int32)
        cxoff, cyoff = np.mean(points_diff, axis=0)
        offsets[structure] = [int(round(cxoff*SCALE)), int(round(cyoff*SCALE)), section]
        
        cv2.polylines(img, [points_diff], isClosed=True, color=(0,0,0), thickness=5)
        cx = centers[structure][0] // SCALE
        cy = centers[structure][1] // SCALE
        
        cv2.circle(img, (cx,cy), 10, (0,0,0), -1)
            
        cv2.polylines(img, [points], isClosed=True, color=colors[structure], thickness=2)
        cv2.putText(img, structure, (cx,cy), font,
                    fontScale, colors[structure], thickness, cv2.LINE_AA)
            
    section_images[section] = img

In [ ]:
# this shows the real structure, it's center of mass and a fake offsetted structure
structure_midpoints = [134,160,177,180,220,268,284,298,330]
section = structure_midpoints[0]
fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(section_images[section], cmap="cool")
plt.title('{} section:{}'.format(animal, section))
plt.show()

In [ ]:
offsets

In [ ]:
origin = np.array(list(centers.values()), dtype=np.float32)
fitted = np.array(list(offsets.values()), dtype=np.float32)
# create a fake structure with a center of mass at:
other_structure = (25000,15000,200)
print(origin.shape, fitted.shape, other_structure)

In [ ]:
origin_centroid = np.mean(origin, axis=0)
fitted_centroid = np.mean(fitted, axis=0)
print(origin_centroid, fitted_centroid, other_structure)

In [ ]:
import affine6p
trans = affine6p.estimate(origin, fitted)
translated = trans.transform(other_structure)
print('translated',translated)
rotated = trans.rotate(other_structure)
print('rotated',rotated)
both = trans.rotate(translated)
print('both',both)

In [ ]:
RMSD, R, T, c = Superpose3D(origin, fitted)

In [ ]:
print(other_structure)
origin_centroid + np.dot(R, other_structure - fitted_centroid)

In [ ]:
# compute the affine transform from the point set
translation, transformation = compute_affine_transform(origin, fitted)
invt = np.linalg.inv(transformation)
offset = -np.dot(invt, translation)
print(other_structure, origin_centroid + np.dot(transformation, other_structure - fitted_centroid))

In [ ]:
colors = 'brgmkbrgmk'
new_points = np.empty_like(origin)
for i in range(len(origin)):
    print('fitted',type(fitted[i]))
    new_points[i] = origin_centroid + np.dot(transformation, fitted[i] - fitted_centroid)
    print('new points', new_points[i])
    print('point %d will move to (%3.1f, %3.1f, %3.1f) to be compared with (%3.1f, %3.1f, %3.1f)' % (
    i, new_points[i, 0], new_points[i, 1], new_points[i, 2], origin[i, 0], origin[i, 1], origin[i, 2]))
    plt.plot(new_points[i, 0], new_points[i, 1], 'x', color=colors[i], markersize=12,
             label='new points' if i == 0 else '')
plt.legend(numpoints=1)
plt.show()


In [ ]:
new_points